In [1]:
import pickle
import sklearn as skl
from scipy.spatial import distance as dist
from imutils import face_utils
import imutils
import dlib
import cv2
import os
from sklearn.svm import SVC
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear

def get_areas(img):
    EYE_AR_THRESH = 0.3
    EYE_AR_CONSEC_FRAMES = 3
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("/media/andrea/Dati2/DLA/ProgettoDLA-Puglisi-/shape_predictor_68_face_landmarks.dat")
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

    imutils.resize(image=img, width=450)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 0)
    # loop over the face detections
    for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0

        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        return ear


In [2]:
import os
import glob


dataset_path = "/media/andrea/Dati2/DLA/"
imagesclosed = [cv2.imread(file) for file in glob.glob(dataset_path+"dataset_B_FacialImages/ClosedFace/" + "*.jpg")]
imagesopen = [cv2.imread(file) for file in glob.glob(dataset_path+"dataset_B_FacialImages/OpenFace/" +"*.jpg")]
train_set = os.listdir(dataset_path)

In [3]:
import numpy as np
#from parfor import parfor
import progressbar
from time import sleep

X = []
Y = []
index = 0

def ears(img, label):
    e = get_areas(img)
    if e is not None:
        X.append(e)
        Y.append(label)
i = 0
bar = progressbar.ProgressBar(maxval=len(imagesclosed), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
bar.start()
for im in imagesclosed:
    bar.update(i+1)
    ears(im, 0)
    i+=1
bar.finish()

[========================================================================] 100%


In [4]:
from sklearn import svm
from sklearn.svm import SVC
import numpy as np
bar = progressbar.ProgressBar(maxval=len(imagesopen), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
bar.start()
i = 0
for im in imagesopen:
    bar.update(i+1)
    ears(im, 1)
    i+=1
bar.finish()

X2 = np.asarray(X).reshape(-1,1)
Y2 = np.asarray(Y)

[========================================================================] 100%


In [22]:
from sklearn import svm
from sklearn.svm import SVC
import numpy as np
clf = svm.SVC()
clf.fit(X2, Y2)
filename = 'svm.sav'
pickle.dump(clf, open(filename, 'wb'))


[0, 3, 12, 27, 48, 75, 108, 147, 192, 243]


In [5]:
from sklearn import svm
import pickle

clf = svm.SVC(C=1000, gamma=0.1)
clf.fit(X2, Y2)
filename = 'svm_C=1000_gamma=0.1.sav'
pickle.dump(clf, open(filename, 'wb'))

